In [31]:
import sys
sys.path.append("../")
import pandas as pd 
import plotly.graph_objects as go 
import datetime as dt 
from plotting import CandlePlot 

In [32]:
from infrastructure.instrument_collection import instrumentCollection as ic


In [33]:
pair ="EUR_USD" 
granularity = "H4"
df = pd.read_pickle(f"../data/{pair}_{granularity}.pkl") 
MA_LIST = [10,20,50,100,200]

In [34]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [35]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c
0,2022-07-17 21:00:00+00:00,1.00853,1.00989,1.00787,1.00916
1,2022-07-18 01:00:00+00:00,1.00917,1.01144,1.00870,1.00916
2,2022-07-18 05:00:00+00:00,1.00917,1.01494,1.00812,1.01306
3,2022-07-18 09:00:00+00:00,1.01306,1.01754,1.01194,1.01368
4,2022-07-18 13:00:00+00:00,1.01370,1.02014,1.01242,1.01670


In [36]:
for ma in  MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(inplace=True, drop=True)


In [37]:
df_plot = df.copy()

In [38]:
df_plot = df_ma.iloc[:500]

In [39]:
traces = [f"MA_{x}" for x in MA_LIST]

In [40]:
cp = CandlePlot(df_plot)
cp.show_plot(line_traces=traces)




In [61]:
MA_S = "MA_50"
MA_L = "MA_200"
BUY = 1
SELL = -1
NONE = 0

In [70]:
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [63]:
df_an.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200
0,2022-09-01 01:00:00+00:00,1.00236,1.00320,1.00092,1.00144,0.998435,1.014479
1,2022-09-01 05:00:00+00:00,1.00142,1.00489,1.00078,1.00372,0.998457,1.014452
2,2022-09-01 09:00:00+00:00,1.00370,1.00469,0.99859,0.99884,0.998358,1.014400
3,2022-09-01 13:00:00+00:00,0.99885,0.99998,0.99108,0.99368,0.998204,1.014303
4,2022-09-01 17:00:00+00:00,0.99368,0.99602,0.99356,0.99470,0.998085,1.014208


In [71]:
df_an['DELTA'] = df_an.MA_50 - df_an.MA_200
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [72]:
def is_trade(row):
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >=0:
        return SELL
    return NONE

In [73]:
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [74]:
df_trades = df_an[df_an.TRADE != NONE].copy()

In [75]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
238,2022-10-26 17:00:00+00:00,1.00765,1.00888,1.00670,1.00852,0.984370,0.984238,0.000132,-0.000472,1
702,2023-02-14 02:00:00+00:00,1.07286,1.07400,1.07255,1.07384,1.077191,1.077214,-0.000023,0.000543,-1
856,2023-03-21 17:00:00+00:00,1.07680,1.07745,1.07564,1.07694,1.066944,1.066899,0.000045,-0.000433,1
1093,2023-05-16 05:00:00+00:00,1.08830,1.08978,1.08629,1.08914,1.096063,1.096156,-0.000093,0.000303,-1
1241,2023-06-19 21:00:00+00:00,1.09226,1.09264,1.09170,1.09204,1.082657,1.082548,0.000109,-0.000409,1


In [76]:
cp = CandlePlot(df_an.iloc[100:1500])
cp.show_plot(line_traces=[MA_S, MA_L])

In [50]:
ic.LoadInstruments("../data")   

In [51]:
ic.instruments_dict[pair]

{'name': 'EUR_USD', 'ins_type': 'CURRENCY', 'displayName': 'EUR/USD', 'pipLocation': 0.0001, 'tradeUnitsPrecision': 0, 'marginRate': 0.03333333333333}

In [77]:
ins_data = ic.instruments_dict[pair] 

In [78]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_50,MA_200,DELTA,DELTA_PREV,TRADE
238,2022-10-26 17:00:00+00:00,1.00765,1.00888,1.00670,1.00852,0.984370,0.984238,0.000132,-0.000472,1
702,2023-02-14 02:00:00+00:00,1.07286,1.07400,1.07255,1.07384,1.077191,1.077214,-0.000023,0.000543,-1
856,2023-03-21 17:00:00+00:00,1.07680,1.07745,1.07564,1.07694,1.066944,1.066899,0.000045,-0.000433,1
1093,2023-05-16 05:00:00+00:00,1.08830,1.08978,1.08629,1.08914,1.096063,1.096156,-0.000093,0.000303,-1
1241,2023-06-19 21:00:00+00:00,1.09226,1.09264,1.09170,1.09204,1.082657,1.082548,0.000109,-0.000409,1


In [80]:
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
df_trades.fillna(0, inplace=True)

In [81]:
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE']

In [82]:
df_trades.GAIN.sum()

np.float64(1680.600000000003)

In [83]:
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [84]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])